<a href="https://colab.research.google.com/github/NguyenThiHongLinh/ML2023/blob/main/Lab_8_20130306_NguyenThiHongLinh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This lab deals with **GridSearchCV** for tuning the hyper-parameters of an estimator and applying vectorization techniques to the **movie reviews dataset** for classification task. 

*   **Deadline: 23:59, 17/4/2023**



# Import libraries

In [1]:
# code
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/MyDrive/ML'

Mounted at /content/gdrive
/content/gdrive/MyDrive/ML


#Task 1. With **iris** dataset
*  1.1. Apply **GridSearchCV** for **SVM** to find the best hyperparameters using the following param_grid.

```
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf','linear']}
```




In [27]:
#code
from sklearn import datasets
from prettytable import PrettyTable
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scipy.stats import mode
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import scipy.cluster.hierarchy as shc
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestRegressor

In [11]:
iris=datasets.load_iris()
X=iris.data
Y=iris.target
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.3, random_state = 4)


In [18]:
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf','linear']}
            
param_grid
svm=SVC()
grid_searchSVM=GridSearchCV(svm, param_grid)
grid_searchSVM.fit(X_train, Y_train)
print("Best parameters: ", grid_searchSVM.best_params_)
print("Best score: ", grid_searchSVM.best_score_)

Best parameters:  {'C': 1, 'gamma': 1, 'kernel': 'rbf'}
Best score:  0.980952380952381


In [13]:
clf=SVC(C= 1, gamma= 1, kernel= 'rbf')
clf.fit(X_train,Y_train)
y_pred=clf.predict(X_test)
print(metrics.classification_report(Y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        21
           1       1.00      0.90      0.95        10
           2       0.93      1.00      0.97        14

    accuracy                           0.98        45
   macro avg       0.98      0.97      0.97        45
weighted avg       0.98      0.98      0.98        45



*  1.2. Apply **GridSearchCV** for **kNN** to find the best hyperparameters using the following param_grid.

```
grid_params = { 'n_neighbors' : [5,7,9,11,13,15],
               'weights' : ['uniform','distance'],
               'metric' : ['minkowski','euclidean','manhattan']}
```
where

    *  **n_neighbors**: Decide the best k based on the values we have computed earlier.
    *  **weights**: Check whether adding weights to the data points is beneficial to the model or not. 'uniform' assigns no weight, while 'distance' weighs points by the inverse of their distances meaning nearer points will have more weight than the farther points.
    *  **metric**: The distance metric to be used will calculating the similarity.


In [16]:
#code
grid_params = { 'n_neighbors' : [5,7,9,11,13,15],
               'weights' : ['uniform','distance'],
               'metric' : ['minkowski','euclidean','manhattan']}
knn=KNeighborsClassifier()
gs=GridSearchCV(knn, grid_params)
gs.fit(X_train, Y_train)
print("Best parameters: ", gs.best_params_)
print("Best score: ", gs.best_score_)

Best parameters:  {'metric': 'minkowski', 'n_neighbors': 9, 'weights': 'uniform'}
Best score:  0.980952380952381


In [21]:
knn = KNeighborsClassifier(**gs.best_params_)
knn.fit(X_train, Y_train)
print("Test set score: ",knn.score(X_test, Y_test))

Test set score:  0.9777777777777777


*  1.3. Apply **GridSearchCV** for **Random Forest** to find the best hyperparameters using the following param_grid.

```
param_grid = {
    'n_estimators': [25, 50, 100, 150],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [3, 6, 9],
    'max_leaf_nodes': [3, 6, 9],
}
```

In [25]:
#code
param_grid = {
    'n_estimators': [25, 50, 100, 150],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [3, 6, 9],
    'max_leaf_nodes': [3, 6, 9],
}
grid_searchRF = GridSearchCV(RandomForestRegressor(), param_grid=param_grid, cv=5)
grid_searchRF.fit(X_train, Y_train)

print("Best parameters:",(grid_searchRF.best_params_))
print("Best cross-validation score:",grid_searchRF.best_score_)

Best parameters: {'max_depth': 9, 'max_features': 'log2', 'max_leaf_nodes': 6, 'n_estimators': 50}
Best cross-validation score: 0.9330085385576654


In [26]:
rf = RandomForestRegressor(n_estimators=grid_searchRF.best_params_['n_estimators'], max_features=grid_searchRF.best_params_['max_features'],max_depth=grid_searchRF.best_params_['max_depth'], max_leaf_nodes=grid_searchRF.best_params_['max_leaf_nodes'])
rf.fit(X_train, Y_train)
print("Test set score: ",rf.score(X_test, Y_test))

Test set score:  0.9790139879700888


*   1.4 Compare the best obtained results from 1.1 to 1.3 (use PrettyTable to dispaly the results)

In [28]:
table = PrettyTable(['Algorithm', 'Best Score', 'Best Parameters'])
table.add_row(['GridSearchCV for SVM', grid_searchSVM.best_score_, grid_searchSVM.best_params_])
table.add_row(['GridSearchCV for kNN', gs.best_score_, gs.best_params_])
table.add_row(['GridSearchCV for Random Forest', grid_searchRF.best_score_, grid_searchRF.best_params_])
print(table)

+--------------------------------+--------------------+-----------------------------------------------------------------------------------+
|           Algorithm            |     Best Score     |                                  Best Parameters                                  |
+--------------------------------+--------------------+-----------------------------------------------------------------------------------+
|      GridSearchCV for SVM      | 0.980952380952381  |                       {'C': 1, 'gamma': 1, 'kernel': 'rbf'}                       |
|      GridSearchCV for kNN      | 0.980952380952381  |          {'metric': 'minkowski', 'n_neighbors': 9, 'weights': 'uniform'}          |
| GridSearchCV for Random Forest | 0.9330085385576654 | {'max_depth': 9, 'max_features': 'log2', 'max_leaf_nodes': 6, 'n_estimators': 50} |
+--------------------------------+--------------------+-----------------------------------------------------------------------------------+


#Task 2. 
For breast cancer dataset (https://tinyurl.com/3vme8hr3) which could be loaded from datasets in sklearn as follows:

```
#Import scikit-learn dataset library
from sklearn import datasets

#Load dataset
cancer = datasets.load_breast_cancer()
```

*   Apply **GridSearchCV** to different classification algorithms such as **SVM, kNN, LogisticRegression, RandomForest**.
*   Compare the results obtained by the best hyperparameters among classification algorithms.

*   2.1. Apply **GridSearchCV** to **SVM** 


In [30]:
data = datasets.load_breast_cancer()
x_train, x_test, y_train, y_test = train_test_split(data['data'], data['target'], test_size=0.3, random_state=1)

In [ ]:
# code
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf','linear']}

grid_searchSVM2 = GridSearchCV(svm, param_grid)

# Fit the GridSearchCV object to the data
grid_searchSVM2.fit(x_train, y_train)

# Print the best parameters and score
print("Best parameters: ", grid_searchSVM2.best_params_)
print("Best score: ", grid_searchSVM2.best_score_)


*   2.2. Apply **GridSearchCV** to **kNN** 

In [ ]:
#code
grid_params = { 'n_neighbors' : [5,7,9,11,13,15],
               'weights' : ['uniform','distance'],
               'metric' : ['minkowski','euclidean','manhattan']}
knn=KNeighborsClassifier()
gs=GridSearchCV(knn, grid_params)
gs.fit(x_train, y_train)
print("Best parameters: ", gs.best_params_)
print("Best score: ", gs.best_score_)

*   2.3. Apply **GridSearchCV** to **LogisticRegression** 

In [ ]:
#code

*   2.4. Apply **GridSearchCV** to **RandomForest** 

In [ ]:
#code

*   2.5. Compare the best obtained results among classification algorithms (use PrettyTable to dispaly the results) 

In [ ]:
#code

#Task 3. 
The dataset consists of **2000 user-created movie reviews** archived on the IMDb(Internet Movie Database). The reviews are equally partitioned into a positive set and a negative set (1000+1000). Each review consists of a plain text file (.txt) and a class label representing the overall user opinion. 
The class attribute has only two values: **pos** (positive) or **neg** (negative).


*   3.1 Importing additional libraries

In [ ]:
import nltk, random
nltk.download('movie_reviews')#download movie reviews dataset
from nltk.corpus import movie_reviews
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import cross_val_score
from collections import Counter
from sklearn.model_selection import train_test_split

*   3.2. Movie reviews information

In [ ]:
#code
print(len(movie_reviews.fileids()))
print(movie_reviews.categories())
print(movie_reviews.words()[:100])
print(movie_reviews.fileids()[:10])

*   3.3. Create dataset from movie reviews

In [ ]:
documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]
random.seed(123)
random.shuffle(documents)

In [ ]:
print('Number of Reviews/Documents: {}'.format(len(documents)))
print('Corpus Size (words): {}'.format(np.sum([len(d) for (d,l) in documents])))
print('Sample Text of Doc 1:')
print('-'*30)
print(' '.join(documents[0][0][:50])) # first 50 words of the first document

In [ ]:
sentiment_distr = Counter([label for (words, label) in documents])
print(sentiment_distr)

*   3.4. Train test split

In [ ]:
train, test = train_test_split(documents, test_size = 0.33, random_state=42)

In [ ]:
## Sentiment Distrubtion for Train and Test
print(Counter([label for (words, label) in train]))
print(Counter([label for (words, label) in test]))

In [ ]:
X_train = [' '.join(words) for (words, label) in train]
X_test = [' '.join(words) for (words, label) in test]
y_train = [label for (words, label) in train]
y_test = [label for (words, label) in test]

*   3.5. Text Vectorization

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

tfidf_vec = TfidfVectorizer(min_df = 10, token_pattern = r'[a-zA-Z]+')
X_train_bow = tfidf_vec.fit_transform(X_train) # fit train
X_test_bow = tfidf_vec.transform(X_test) # transform test

*   3.6. Apply **SVM** with **GridSearchCV** 

In [ ]:
#code

*   3.7. Apply **RandomForest** with **GridSearchCV** 

In [ ]:
#code

*   3.8. Apply **kNN** with **GridSearchCV** 

In [ ]:
#code

*   3.9. Apply **LogisticRegression** with **GridSearchCV** 

In [ ]:
#code

*   3.10. Compare the best obtained results among classification algorithms (use PrettyTable to dispaly the results) 

#Finally,
Save a copy in your Github. Remember renaming the notebook.